In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re
import pickle

from vietunits.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [2]:
def create_sort(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s|Thủ đô\s', '', text, flags=re.IGNORECASE)
        else:
            text = re.sub(r'^Phường\s|Đặc khu\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text

district_type_acronym = {
    'Quận': 'q',
    'Thị xã': 'tx',
    'Thành phố': 'tp',
    'Huyện': 'h',
}
ward_type_acronym = {
    'Phường': 'p',
    'Đặc khu': 'dk',
    'Xã': 'x'
}
def create_keywords(row, level=1):
    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    else:
        keywords.append(row['wardKey'])
        if not row['wardShortKeyDuplicated']:
            keywords.append(row['wardShortKey'])
        else:
            keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
            keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]} {row['wardShortKey']}"))

        if pd.notnull(row['wardAlias']):
            aliases = json.loads(row['wardAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)

In [3]:
df_convert = pd.read_csv(BASE_DIR / 'data/danhmuc_and_sapnhap_has_default_new_ward.csv')
col_name_map = {
    'newProvinceCode': 'provinceCode',
    'newWardCode': 'wardCode',
    'newWardType': 'wardType',
    'newProvince': 'province',
    'newWard': 'wardShort',
}
df = df_convert[col_name_map.keys()].drop_duplicates().reset_index(drop=True).rename(columns=col_name_map)

df['wardType'] = df['wardType'].str.capitalize()
df['ward'] = df['wardType'] + ' ' + df['wardShort']

In [4]:
# ENRICH DATA
unit_cols = ['province', 'ward']
level_map = {
    'province': 1,
    'ward': 2
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    df[f"{col}Short"] = df[col].apply(create_sort, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

In [5]:
# -- CREATE ALIAS
# Khởi tạo cột alias rỗng
for col in ['province', 'ward']:
    df[f"{col}Alias"] = np.nan

# Province alias data
province_alias_data = {
    'thudohanoi': ['hn'],
    'thanhphohochiminh': ['hcm'],
}

# Ward alias data (theo từng province)
ward_alias_data = {
    # 'thanhphohochiminh': {
    #     'wardkey': ['quan9', 'quan2']
    # }
}


# Gán provinceAlias
for key, value in province_alias_data.items():
    df.loc[df['provinceKey'] == key, 'provinceAlias'] = json.dumps(value)

# Gán wardAlias
for province_key, ward_data in ward_alias_data.items():
    for ward_key, value in ward_data.items():
        df.loc[
            (df['provinceKey'] == province_key) & (df['wardKey'] == ward_key), 'wardAlias'] = json.dumps(value)

In [6]:
# Check ward key
count_ward_key = df.groupby(['province', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_key = count_ward_key[count_ward_key['count']>1].copy()
duplicated_ward_key['wardKeyDuplicated'] = True
duplicated_ward_key.drop(columns=['count'], inplace=True)

df = pd.merge(df, duplicated_ward_key, on=['province', 'wardKey'], how='left')
df['wardKeyDuplicated'].fillna(False, inplace=True)

In [7]:
duplicated_ward_key

,province,wardKey,wardKeyDuplicated
416,Thành phố Hồ Chí Minh,xathanhan,True
199,Thành phố Hải Phòng,xacamgiang,True
2618,Tỉnh Thái Nguyên,xavanlang,True
2140,Tỉnh Quảng Ngãi,xasonha,True
2829,Tỉnh Tây Ninh,xatanthanh,True
3168,Tỉnh Đồng Nai,xalocthanh,True
2084,Tỉnh Quảng Ngãi,xabato,True
3299,Tỉnh Đồng Tháp,xatanthanh,True


In [8]:
df['wardKey'] = np.where(df['wardKeyDuplicated']==True, df['ward'].apply(key_normalize, args=([], False)), df['wardKey'])
df['wardShortKey'] = np.where(df['wardKeyDuplicated']==True, df['wardShort'].apply(key_normalize, args=([], False)), df['wardShortKey'])

In [9]:
df[df['wardKeyDuplicated']==True]

,provinceCode,wardCode,wardType,province,wardShort,ward,provinceShort,provinceKey,provinceShortKey,wardKey,wardShortKey,provinceAlias,wardAlias,wardKeyDuplicated
287,10,652.0,Xã,Tỉnh Thái Nguyên,Văn Lang,Xã Văn Lang,Thái Nguyên,tinhthainguyen,thainguyen,xãvănlang,vănlang,NaN,NaN,True
671,10,653.0,Xã,Tỉnh Thái Nguyên,Văn Lăng,Xã Văn Lăng,Thái Nguyên,tinhthainguyen,thainguyen,xãvănlăng,vănlăng,NaN,NaN,True
1059,4,1052.0,Xã,Thành phố Hải Phòng,Cẩm Giang,Xã Cẩm Giang,Hải Phòng,thanhphohaiphong,haiphong,xãcẩmgiang,cẩmgiang,NaN,NaN,True
1060,4,1053.0,Xã,Thành phố Hải Phòng,Cẩm Giàng,Xã Cẩm Giàng,Hải Phòng,thanhphohaiphong,haiphong,xãcẩmgiàng,cẩmgiàng,NaN,NaN,True
1969,22,2037.0,Xã,Tỉnh Quảng Ngãi,Sơn Hà,Xã Sơn Hà,Quảng Ngãi,tinhquangngai,quangngai,xãsơnhà,sơnhà,NaN,NaN,True
1970,22,2038.0,Xã,Tỉnh Quảng Ngãi,Sơn Hạ,Xã Sơn Hạ,Quảng Ngãi,tinhquangngai,quangngai,xãsơnhạ,sơnhạ,NaN,NaN,True
1992,22,1973.0,Xã,Tỉnh Quảng Ngãi,Ba Tơ,Xã Ba Tơ,Quảng Ngãi,tinhquangngai,quangngai,xãbatơ,batơ,NaN,NaN,True
1998,22,1972.0,Xã,Tỉnh Quảng Ngãi,Ba Tô,Xã Ba Tô,Quảng Ngãi,tinhquangngai,quangngai,xãbatô,batô,NaN,NaN,True
2478,28,2630.0,Xã,Tỉnh Đồng Nai,Lộc Thạnh,Xã Lộc Thạnh,Đồng Nai,tinhdongnai,dongnai,xãlộcthạnh,lộcthạnh,NaN,NaN,True
2482,28,2629.0,Xã,Tỉnh Đồng Nai,Lộc Thành,Xã Lộc Thành,Đồng Nai,tinhdongnai,dongnai,xãlộcthành,lộcthành,NaN,NaN,True


In [10]:
# Check ward short key
count_ward_short_key = df.groupby(['province', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)

df = pd.merge(df, duplicated_ward_short_key, on=['province', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)

In [11]:
duplicated_ward_short_key

,province,wardShortKey,wardShortKeyDuplicated
2398,Tỉnh Thanh Hóa,dongtien,True
3269,Tỉnh Đồng Tháp,mytho,True


In [12]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

In [13]:
df

,provinceCode,wardCode,wardType,province,wardShort,ward,provinceShort,provinceKey,provinceShortKey,wardKey,wardShortKey,provinceAlias,wardAlias,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,wardKeywords
0,1,42.0,Phường,Thủ đô Hà Nội,Hồng Hà,Phường Hồng Hà,Hà Nội,thudohanoi,hanoi,phuonghongha,hongha,"[""hn""]",NaN,False,False,"[""thudohanoi"", ""hanoi"", ""hn""]","[""phuonghongha"", ""hongha""]"
1,1,3.0,Phường,Thủ đô Hà Nội,Ba Đình,Phường Ba Đình,Hà Nội,thudohanoi,hanoi,phuongbadinh,badinh,"[""hn""]",NaN,False,False,"[""thudohanoi"", ""hanoi"", ""hn""]","[""phuongbadinh"", ""badinh""]"
2,1,61.0,Phường,Thủ đô Hà Nội,Ngọc Hà,Phường Ngọc Hà,Hà Nội,thudohanoi,hanoi,phuongngocha,ngocha,"[""hn""]",NaN,False,False,"[""thudohanoi"", ""hanoi"", ""hn""]","[""phuongngocha"", ""ngocha""]"
3,1,30.0,Phường,Thủ đô Hà Nội,Giảng Võ,Phường Giảng Võ,Hà Nội,thudohanoi,hanoi,phuonggiangvo,giangvo,"[""hn""]",NaN,False,False,"[""thudohanoi"", ""hanoi"", ""hn""]","[""phuonggiangvo"", ""giangvo""]"
4,1,39.0,Phường,Thủ đô Hà Nội,Hoàn Kiếm,Phường Hoàn Kiếm,Hà Nội,thudohanoi,hanoi,phuonghoankiem,hoankiem,"[""hn""]",NaN,False,False,"[""thudohanoi"", ""hanoi"", ""hn""]","[""phuonghoankiem"", ""hoankiem""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,34,3331.0,Xã,Tỉnh Cà Mau,Phú Tân,Xã Phú Tân,Cà Mau,tinhcamau,camau,xaphutan,phutan,NaN,NaN,False,False,"[""tinhcamau"", ""camau""]","[""xaphutan"", ""phutan""]"
3317,34,3324.0,Xã,Tỉnh Cà Mau,Nguyễn Việt Khái,Xã Nguyễn Việt Khái,Cà Mau,tinhcamau,camau,xanguyenvietkhai,nguyenvietkhai,NaN,NaN,False,False,"[""tinhcamau"", ""camau""]","[""xanguyenvietkhai"", ""nguyenvietkhai""]"
3318,34,3337.0,Xã,Tỉnh Cà Mau,Tân Ân,Xã Tân Ân,Cà Mau,tinhcamau,camau,xatanan,tanan,NaN,NaN,False,False,"[""tinhcamau"", ""camau""]","[""xatanan"", ""tanan""]"
3319,34,3327.0,Xã,Tỉnh Cà Mau,Phan Ngọc Hiển,Xã Phan Ngọc Hiển,Cà Mau,tinhcamau,camau,xaphanngochien,phanngochien,NaN,NaN,False,False,"[""tinhcamau"", ""camau""]","[""xaphanngochien"", ""phanngochien""]"


In [14]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
    }

In [15]:
def build_province_ward_dict(df):
    result = {}

    # Nhóm theo provinceKey
    grouped = df.groupby('provinceKey')

    for province_key, group in grouped:
        ward_dict = {}

        for _, row in group.iterrows():
            ward_key = row['wardKey']
            ward_keywords = json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else []
            ward_dict[ward_key] = {
                'wardKeywords': ward_keywords,
                'ward': row['ward'],
                'wardType': row['wardType'],
                'wardShort': row['wardShort'],
            }

        result[province_key] = ward_dict

    return result

df_ward_no_accented = df[df['wardKeyDuplicated'] == False]
df_ward_accented = df[df['wardKeyDuplicated'] == True]

DICT_PROVINCE_WARD_NO_ACCENTED = build_province_ward_dict(df_ward_no_accented)
DICT_PROVINCE_WARD_ACCENTED = build_province_ward_dict(df_ward_accented)

In [16]:
# Lấy danh sách các wardShortKey không trùng với bất kỳ hàng nào khác (trừ chính nó) và không trùng provinceShortKey
province_short_keys = set(df['provinceShortKey'].unique())
ward_short_key_counts = df['wardShortKey'].value_counts()

# Điều kiện wardUnique: xuất hiện đúng 1 lần và không nằm trong province_short_keys
df['wardUnique'] = df['wardShortKey'].map(ward_short_key_counts) == 1
df['wardUnique'] &= ~df['wardShortKey'].isin(province_short_keys)

# Chia nhỏ theo wardKeyDuplicated
df_ward_unique = df[df['wardUnique']]
df_ward_unique_accented = df_ward_unique[df_ward_unique['wardKeyDuplicated']]
df_ward_unique_no_accented = df_ward_unique[~df_ward_unique['wardKeyDuplicated']]

# Tạo dict bằng dictionary comprehension
DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED = {
    row['wardKey']: {
        'wardKeywords': json.loads(row['wardKeywords']),
        'provinceKey': row['provinceKey']
    }
    for _, row in df_ward_unique_no_accented.iterrows()
}

DICT_UNIQUE_WARD_PROVINCE_ACCENTED = {
    row['wardKey']: {
        'wardKeywords': json.loads(row['wardKeywords']),
        'provinceKey': row['provinceKey']
    }
    for _, row in df_ward_unique_accented.iterrows()
}

In [18]:
pickle_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_WARD_NO_ACCENTED': DICT_PROVINCE_WARD_NO_ACCENTED,
    'DICT_PROVINCE_WARD_ACCENTED': DICT_PROVINCE_WARD_ACCENTED,
    'DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED': DICT_UNIQUE_WARD_PROVINCE_NO_ACCENTED,
    'DICT_UNIQUE_WARD_PROVINCE_ACCENTED': DICT_UNIQUE_WARD_PROVINCE_ACCENTED
}

In [19]:
with open(BASE_DIR / 'vietunits/data/v34provinces/pickle_data.pkl', 'wb') as f:
    pickle.dump(pickle_data, f)